### Kaggle Score : RMSE : 20

In [1]:
import tensorflow as tf
# tf.logging.set_verbosity(tf.logging.ERROR)

import numpy as np
import keras
from PIL import Image
# from IPython.display import display, HTML
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D,Dropout, Flatten, Dense
from keras import regularizers
# from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder # one-hot encoding for age
from keras.utils.np_utils import to_categorical
from tensorflow.keras.optimizers import SGD

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
import cv2
import os

In [2]:
pawpularity_path = '../input/petfinder-pawpularity-score';
df = pd.read_csv(pawpularity_path+'/train.csv')
df.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72


In [3]:
df['Id'] = df['Id'] + '.jpg';
df['Id']

0       0007de18844b0dbbb5e1f607da0606e0.jpg
1       0009c66b9439883ba2750fb825e1d7db.jpg
2       0013fd999caf9a3efe1352ca1b0d937e.jpg
3       0018df346ac9c1d8413cfcc888ca8246.jpg
4       001dc955e10590d3ca4673f034feeef2.jpg
                        ...                 
9907    ffbfa0383c34dc513c95560d6e1fdb57.jpg
9908    ffcc8532d76436fc79e50eb2e5238e45.jpg
9909    ffdf2e8673a1da6fb80342fa3b119a20.jpg
9910    fff19e2ce11718548fa1c5d039a5192a.jpg
9911    fff8e47c766799c9e12f3cb3d66ad228.jpg
Name: Id, Length: 9912, dtype: object

In [4]:
df.drop([ 'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
       'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'],axis = 1,inplace = True)

In [5]:
X = []
Y = []
width = 100
height = 100

for index, row in df.iterrows():
    file_path = pawpularity_path+'/train/'+ row.Id
    image = Image.open(file_path)
    image = image.convert('RGB')
    image = image.resize((width, height))
    X.append(np.array(image))
    Y.append(row.Pawpularity)

In [6]:
# Normalise input data to range [0,1]
X = np.array(X)
X = X.astype('float32')
X /= 255.0
# confirm the normalization
print('Min: %.3f, Max: %.3f' % (X.min(), X.max()))

Y = np.array(Y)

Min: 0.000, Max: 1.000


In [7]:
test_size = 0.3
seed = 42
X_train, X_test, Y_train, Y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=test_size, 
                                                    random_state=seed,
                                                    shuffle=True,
                                                    stratify=Y)

X_test, X_val, Y_test, Y_val = train_test_split(X_test, Y_test, test_size=0.5)

In [8]:
# Apply Image Augmentation

train_datagen = ImageDataGenerator(
    shear_range = 0.2, # random application of shearing
    zoom_range = 0.2,
    horizontal_flip = True) # randomly flipping half of the images horizontally

test_datagen = ImageDataGenerator(
)

In [9]:
from keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping
from timeit import default_timer as timer

class TimingCallback(Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

early_stopping = EarlyStopping(
                                patience=5, # wait for 5 epochs
                                min_delta = 0.01, # if in 5 epochs the loss function doesn't inrease (for accuracy) 
                                               # or decrease (for val_loss) by 1%, then stop
                                verbose=1, # print the training epoch on which training was stopped
                                mode = 'min',
                                monitor='val_loss')

reduce_learning_rate = ReduceLROnPlateau(
                                    monitor="val_loss",
                                    patience=3, # if val_loss plateaus for 3 epochs such that it doesn't see 
                                                # an improvement of size = epsilon
                                    episilon= 0.01,
                                    factor=0.1,  # then we reduce the learning rate by a factor of 0.1
                                    cooldown = 4, # and we wait for 4 epochs before we restart again
                                    verbose=1)

time_callback = TimingCallback()

        
# hyperparameters
lr = 0.01
epochs = 30 # setting it to a low number since this is for Kaggle, ideally you should set this to a higher number ~ 100 so that the model overfits training data, and then apply the elbow methods to select the best params on validaion data
batch_size = 128
results = {}
input_shape =[width,height]
num_of_ages = 100

In [10]:
from keras.applications.mobilenet import MobileNet
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, BatchNormalization
from keras.models import Sequential
base_mobilenet_model = MobileNet(input_shape =  (input_shape[0],input_shape[1], 3), 
                                 include_top = False, 
                                 weights = None)
mobilenet_model = Sequential()
mobilenet_model.add(BatchNormalization(input_shape = (input_shape[0],input_shape[1], 3)))
mobilenet_model.add(base_mobilenet_model)
mobilenet_model.add(BatchNormalization())
mobilenet_model.add(GlobalAveragePooling2D())
mobilenet_model.add(Dropout(0.5))
mobilenet_model.add(Dense(1,activation='linear')) 



mobilenet_model.compile(optimizer = 'adam', loss = 'mean_absolute_error',
                           metrics = ["mean_absolute_error"])

# mobilenet_model.summary()

# Fit model on training data
mobilenet_model_history = mobilenet_model.fit_generator(
            train_datagen.flow(X_train,Y_train,
                         batch_size = batch_size), 
            validation_data = (X_val,Y_val),
            epochs = epochs,
            callbacks = [
                         reduce_learning_rate,
                         early_stopping,
                         time_callback
                        ],
            verbose=True)


User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_INIT_AT_FORK=FALSE
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_P

Epoch 1/30
55/55 [==============================] - 109s 2s/step - loss: 36.6695 - mean_absolute_error: 36.6695 - val_loss: 34.7690 - val_mean_absolute_error: 34.7690
Epoch 2/30
55/55 [==============================] - 103s 2s/step - loss: 29.6512 - mean_absolute_error: 29.6512 - val_loss: 23.7338 - val_mean_absolute_error: 23.7338
Epoch 3/30
55/55 [==============================] - 104s 2s/step - loss: 17.8588 - mean_absolute_error: 17.8588 - val_loss: 15.3930 - val_mean_absolute_error: 15.3930
Epoch 4/30
55/55 [==============================] - 104s 2s/step - loss: 15.2002 - mean_absolute_error: 15.2002 - val_loss: 15.2779 - val_mean_absolute_error: 15.2779
Epoch 5/30
55/55 [==============================] - 103s 2s/step - loss: 15.1138 - mean_absolute_error: 15.1138 - val_loss: 15.3548 - val_mean_absolute_error: 15.3548
Epoch 6/30
55/55 [==============================] - 103s 2s/step - loss: 14.9955 - mean_absolute_error: 14.9955 - val_loss: 15.7237 - val_mean_absolute_error: 15.723

In [11]:
# MobileNet

mobilenet_model_test_results = mobilenet_model.evaluate(X_test, Y_test, batch_size=128)
print(dict(zip(mobilenet_model.metrics_names, mobilenet_model_test_results)))

12/12 [==============================] - 4s 310ms/step - loss: 14.8141 - mean_absolute_error: 14.8141
{'loss': 14.814105987548828, 'mean_absolute_error': 14.814105987548828}


In [12]:
df_test = pd.read_csv(pawpularity_path+'/test.csv')
Id = df_test["Id"].copy()

In [13]:
df_test["Id"] = df_test["Id"].apply(lambda x : x + ".jpg")

In [14]:
X = []
Y = []
width = 100
height = 100

for index, row in df_test.iterrows():
    file_path = pawpularity_path+'/test/'+ row.Id
    image = Image.open(file_path)
    image = image.convert('RGB')
    image = image.resize((width, height))
    X.append(np.array(image))
#     Y.append(row.Pawpularity)

In [15]:
# Normalise input data to range [0,1]
X = np.array(X)
X = X.astype('float32')
X /= 255.0
# confirm the normalization
print('Min: %.3f, Max: %.3f' % (X.min(), X.max()))

# Y = np.array(Y)

Min: 0.000, Max: 1.000


In [16]:
predicted_ages = mobilenet_model.predict(X)

In [17]:
predicted_ages

array([[28.85332 ],
       [28.85919 ],
       [28.862677],
       [28.85764 ],
       [28.862047],
       [28.86506 ],
       [28.853378],
       [28.86103 ]], dtype=float32)

In [18]:
submission_file = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')

submission_file['Pawpularity'] = predicted_ages

submission_file.to_csv("submission.csv", index=False)